In [11]:
import os
# Set PyTorch memory allocation configuration
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

import torch
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm
from src.data_loader import MRIDataset
from src.tsgan import UNet3DGenerator, Discriminator3D
from src.utils.logger import get_logger, get_tensorboard_writer
from src.utils.visualization import visualize_mri_comparison
import numpy as np
import torchvision.models as models
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.nn as nn
from torch.cuda.amp import autocast, GradScaler


# === Configs ===
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DATA_DIR = r"C:\Users\OMEN\Desktop\MRI_Synthesis_Project\MRI_SYNTHESIS_PROJECT\data\preprocessed"
EPOCHS = 50
BATCH_SIZE = 1
LR = 5e-5
VAL_SPLIT = 0.15
GRADIENT_ACCUMULATION_STEPS = 4  # Added gradient accumulation

# === Logging ===
logger = get_logger()
writer = get_tensorboard_writer()

# === Dataset & DataLoader Split ===
full_dataset = MRIDataset(base_dir=DATA_DIR)
val_size = int(VAL_SPLIT * len(full_dataset))
train_size = len(full_dataset) - val_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

# === Model Setup ===
generator = UNet3DGenerator().to(DEVICE)
discriminator = Discriminator3D().to(DEVICE)

# === VGG16 for Perceptual Loss (Reduced Size) ===
vgg = models.vgg16(pretrained=True).features[:4].eval().to(DEVICE)  # Reduced from [:9] to [:4]
for p in vgg.parameters():
    p.requires_grad = False

vgg_normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])

g_opt = torch.optim.Adam(generator.parameters(), lr=LR, betas=(0.5, 0.999))
d_opt = torch.optim.Adam(discriminator.parameters(), lr=LR, betas=(0.5, 0.999))

l1_loss = nn.L1Loss()
bce = nn.BCEWithLogitsLoss()

# === Mixed Precision Setup ===
scaler = GradScaler()

# === Output Directories ===
os.makedirs("outputs/fake_vs_real_2", exist_ok=True)
os.makedirs("outputs/checkpointss", exist_ok=True)

# === Dice Loss Function ===
def dice_loss(pred, target, smooth=1e-5):
    intersection = (pred * target).sum()
    return 1 - (2. * intersection + smooth) / (pred.sum() + target.sum() + smooth)

# === Training Loop ===
logger.info(f"Starting training for {EPOCHS} epochs...")

for epoch in range(EPOCHS):
    generator.train()
    discriminator.train()
    
    # Reset optimizers at the beginning of each epoch when using gradient accumulation
    d_opt.zero_grad()
    g_opt.zero_grad()
    
    for i, (pret1, cet1, seg) in enumerate(tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}")):
        pret1, cet1, seg = pret1.to(DEVICE), cet1.to(DEVICE), seg.to(DEVICE)

        # === Train Discriminator with Mixed Precision ===
        with autocast():
            real_output = discriminator(pret1, cet1)
            fake_cet1 = generator(pret1, seg)
            fake_output = discriminator(pret1, fake_cet1.detach())

            d_loss_real = bce(real_output, torch.ones_like(real_output))
            d_loss_fake = bce(fake_output, torch.zeros_like(fake_output))
            d_loss = (d_loss_real + d_loss_fake) / GRADIENT_ACCUMULATION_STEPS
        
        # Scale the loss and do backward pass
        scaler.scale(d_loss).backward()
        
        # Only step optimizer after accumulating gradients
        if (i + 1) % GRADIENT_ACCUMULATION_STEPS == 0 or (i + 1 == len(train_loader)):
            scaler.step(d_opt)
            scaler.update()
            d_opt.zero_grad()

        # === Train Generator with Mixed Precision ===
        with autocast():
            fake_output = discriminator(pret1, fake_cet1)
            g_loss_adv = bce(fake_output, torch.ones_like(fake_output))
            g_loss_l1 = l1_loss(fake_cet1, cet1)

            # === Perceptual Loss (VGG16) ===
            fake_cet1_vgg = vgg(vgg_normalize(fake_cet1))
            cet1_vgg = vgg(vgg_normalize(cet1))
            perceptual_loss = l1_loss(fake_cet1_vgg, cet1_vgg)

            # === Dice Loss ===
            dice_loss_value = dice_loss(fake_cet1, cet1)

            # Total Generator Loss with gradient accumulation
            g_loss = (g_loss_adv + 100 * g_loss_l1 + 10 * perceptual_loss + 5 * dice_loss_value) / GRADIENT_ACCUMULATION_STEPS
        
        # Scale the loss and do backward pass
        scaler.scale(g_loss).backward()
        
        # Only step optimizer after accumulating gradients
        if (i + 1) % GRADIENT_ACCUMULATION_STEPS == 0 or (i + 1 == len(train_loader)):
            scaler.step(g_opt)
            scaler.update()
            g_opt.zero_grad()

        # === Logging ===
        writer.add_scalar("Loss/Discriminator", d_loss.item() * GRADIENT_ACCUMULATION_STEPS, epoch * len(train_loader) + i)
        writer.add_scalar("Loss/Generator", g_loss.item() * GRADIENT_ACCUMULATION_STEPS, epoch * len(train_loader) + i)

        # === Save visual slices - reduced frequency to save memory ===
        if i % 10 == 0:  # Changed from 5 to 10
            real_slice = cet1[0, 0, :, :, cet1.shape[4] // 2].detach().cpu().numpy()
            fake_slice = fake_cet1[0, 0, :, :, fake_cet1.shape[4] // 2].detach().cpu().numpy()
            plot_path = f"outputs/fake_vs_real_2/fake_real_epoch{epoch}_step{i}.png"
            visualize_mri_comparison(real_slice, fake_slice, plot_path, title="Real vs Fake CeT1")

        # Explicitly clear cache to save memory
        if i % 5 == 0:
            torch.cuda.empty_cache()

        if i % 10 == 0:
            # Multiply by accumulation steps to show the true loss value
            print(f"Step {i}: G_Loss = {g_loss.item() * GRADIENT_ACCUMULATION_STEPS:.4f}, D_Loss = {d_loss.item() * GRADIENT_ACCUMULATION_STEPS:.4f}")

    logger.info(f"Epoch [{epoch+1}/{EPOCHS}] - G_Loss: {g_loss.item() * GRADIENT_ACCUMULATION_STEPS:.4f}, D_Loss: {d_loss.item() * GRADIENT_ACCUMULATION_STEPS:.4f}")

    # Save checkpoint after every epoch
    torch.save(generator.state_dict(), f"outputs/checkpointss/generator_epoch{epoch+1}.pth")
    torch.save(discriminator.state_dict(), f"outputs/checkpointss/discriminator_epoch{epoch+1}.pth")
    
    # Clear memory between epochs
    torch.cuda.empty_cache()

logger.info("Training complete.")

FileNotFoundError: [Errno 2] No such file or directory: 'c:\\Users\\OMEN\\Desktop\\MRI_Synthesis_Project\\MRI_SYNTHESIS_PROJECT\\notebooks\\outputs\\logs\\log.txt'

chat gpt 


In [20]:
import os
# Set PyTorch memory allocation configuration
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

import torch
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm
from src.data_loader import MRIDataset
from src.tsgan import UNet3DGenerator, Discriminator3D
from src.utils.logger import get_logger, get_tensorboard_writer
from src.utils.visualization import visualize_mri_comparison
import numpy as np
import torchvision.models as models
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.nn as nn
from torch.cuda.amp import autocast, GradScaler

# === Configs ===
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DATA_DIR = r"C:\Users\OMEN\Desktop\MRI_Synthesis_Project\MRI_SYNTHESIS_PROJECT\data\preprocessed"
EPOCHS = 50
BATCH_SIZE = 1
LR = 5e-5
VAL_SPLIT = 0.15
GRADIENT_ACCUMULATION_STEPS = 4

# === Logging ===
log_dir = r"C:\Users\OMEN\Desktop\MRI_Synthesis_Project\MRI_SYNTHESIS_PROJECT\outputs"
logger = get_logger(log_dir=log_dir)
writer = get_tensorboard_writer(log_dir=log_dir)

# === Dataset & DataLoader ===
full_dataset = MRIDataset(DATA_DIR)
val_size = int(VAL_SPLIT * len(full_dataset))
train_size = len(full_dataset) - val_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

# === Model Setup ===
generator = UNet3DGenerator().to(DEVICE)
discriminator = Discriminator3D().to(DEVICE)

# === VGG16 for Perceptual Loss ===
vgg = models.vgg16(pretrained=True).features[:4].eval().to(DEVICE)
for p in vgg.parameters():
    p.requires_grad = False

vgg_normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])

g_opt = torch.optim.Adam(generator.parameters(), lr=LR, betas=(0.5, 0.999))
d_opt = torch.optim.Adam(discriminator.parameters(), lr=LR, betas=(0.5, 0.999))

l1_loss = nn.L1Loss()
bce = nn.BCEWithLogitsLoss()
scaler = GradScaler()

# === Output Directories ===
os.makedirs(os.path.join(log_dir, "fake_vs_real_2"), exist_ok=True)
os.makedirs(os.path.join(log_dir, "checkpointss"), exist_ok=True)

# === Dice Loss ===
def dice_loss(pred, target, smooth=1e-5):
    intersection = (pred * target).sum()
    return 1 - (2. * intersection + smooth) / (pred.sum() + target.sum() + smooth)

# === Training Loop ===
logger.info(f"Starting training for {EPOCHS} epochs...")

for epoch in range(EPOCHS):
    generator.train()
    discriminator.train()
    d_opt.zero_grad()
    g_opt.zero_grad()

    for i, (pret1, cet1, seg) in enumerate(tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}")):
        if isinstance(pret1, str) or isinstance(cet1, str) or isinstance(seg, str):
            raise TypeError("MRIDataset must return tensors, not strings. Please fix MRIDataset __getitem__")

        pret1, cet1, seg = pret1.to(DEVICE), cet1.to(DEVICE), seg.to(DEVICE)

        # === Train Discriminator ===
        with autocast():
            real_output = discriminator(pret1, cet1)
            fake_cet1 = generator(pret1, seg)
            fake_output = discriminator(pret1, fake_cet1.detach())

            d_loss_real = bce(real_output, torch.ones_like(real_output))
            d_loss_fake = bce(fake_output, torch.zeros_like(fake_output))
            d_loss = (d_loss_real + d_loss_fake) / GRADIENT_ACCUMULATION_STEPS

        scaler.scale(d_loss).backward()

        if (i + 1) % GRADIENT_ACCUMULATION_STEPS == 0 or (i + 1 == len(train_loader)):
            scaler.step(d_opt)
            scaler.update()
            d_opt.zero_grad()

        # === Train Generator ===
        with autocast():
            fake_output = discriminator(pret1, fake_cet1)
            g_loss_adv = bce(fake_output, torch.ones_like(fake_output))
            g_loss_l1 = l1_loss(fake_cet1, cet1)

            fake_cet1_vgg = vgg(vgg_normalize(fake_cet1))
            cet1_vgg = vgg(vgg_normalize(cet1))
            perceptual_loss = l1_loss(fake_cet1_vgg, cet1_vgg)
            dice_loss_value = dice_loss(fake_cet1, cet1)

            g_loss = (g_loss_adv + 100 * g_loss_l1 + 10 * perceptual_loss + 5 * dice_loss_value) / GRADIENT_ACCUMULATION_STEPS

        scaler.scale(g_loss).backward()

        if (i + 1) % GRADIENT_ACCUMULATION_STEPS == 0 or (i + 1 == len(train_loader)):
            scaler.step(g_opt)
            scaler.update()
            g_opt.zero_grad()

        writer.add_scalar("Loss/Discriminator", d_loss.item() * GRADIENT_ACCUMULATION_STEPS, epoch * len(train_loader) + i)
        writer.add_scalar("Loss/Generator", g_loss.item() * GRADIENT_ACCUMULATION_STEPS, epoch * len(train_loader) + i)

        if i % 10 == 0:
            real_slice = cet1[0, 0, :, :, cet1.shape[4] // 2].detach().cpu().numpy()
            fake_slice = fake_cet1[0, 0, :, :, fake_cet1.shape[4] // 2].detach().cpu().numpy()
            plot_path = os.path.join(log_dir, f"fake_vs_real_2/fake_real_epoch{epoch}_step{i}.png")
            visualize_mri_comparison(real_slice, fake_slice, plot_path, title="Real vs Fake CeT1")

        if i % 5 == 0:
            torch.cuda.empty_cache()

        if i % 10 == 0:
            print(f"Step {i}: G_Loss = {g_loss.item() * GRADIENT_ACCUMULATION_STEPS:.4f}, D_Loss = {d_loss.item() * GRADIENT_ACCUMULATION_STEPS:.4f}")

    logger.info(f"Epoch [{epoch+1}/{EPOCHS}] - G_Loss: {g_loss.item() * GRADIENT_ACCUMULATION_STEPS:.4f}, D_Loss: {d_loss.item() * GRADIENT_ACCUMULATION_STEPS:.4f}")

    torch.save(generator.state_dict(), os.path.join(log_dir, f"checkpointss/generator_epoch{epoch+1}.pth"))
    torch.save(discriminator.state_dict(), os.path.join(log_dir, f"checkpointss/discriminator_epoch{epoch+1}.pth"))
    torch.cuda.empty_cache()

logger.info("Training complete.")


C:\Users\OMEN\AppData\Local\Temp\ipykernel_5164\1399793329.py:57: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
Epoch 1/50:   0%|          | 0/48 [00:00<?, ?it/s]


TypeError: MRIDataset must return tensors, not strings. Please fix MRIDataset __getitem__

In [8]:
import torch
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm
import os
from src.dataset import MRIDataset
from src.tsgan import UNet3DGenerator, Discriminator3D
from src.utils.logger import get_logger, get_tensorboard_writer
from src.utils.visualization import visualize_mri_comparison
import numpy as np
import torchvision.models as models
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.nn as nn
from torch.cuda.amp import autocast, GradScaler


In [10]:
import sys
import os

# Add root path to sys.path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))  # Adjust if needed
sys.path.append(project_root)

# Now import your modules
from src.dataset import MRIDataset
from src.tsgan import UNet3DGenerator, Discriminator3D
from src.utils.logger import get_logger, get_tensorboard_writer
from src.utils.visualization import visualize_mri_comparison


In [31]:
from src.data_loader import MRIDataset

dataset = MRIDataset(r"C:\Users\OMEN\Desktop\MRI_Synthesis_Project\MRI_SYNTHESIS_PROJECT\data\preprocessed")
train_loader = DataLoader(dataset, batch_size=1, shuffle=True)

pret1, cet1, seg = dataset[0]
print(type(pret1), pret1.shape)  


AttributeError: 'str' object has no attribute 'shape'